# Segmenting and Clustering Neighborhoods in Toronto


In [77]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import folium
import requests
from tqdm import tqdm
from collections import deque
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Load data

I load the data generated in the previous notebook and work with only boroughs that contain the word Toronto  

In [78]:
data = pd.read_csv("data.csv").drop(['Unnamed: 0'], axis=1)
TorontoData = data[data.Borough.str.contains("Toronto")]
TorontoData.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
10,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
12,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [79]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoData['Borough'].unique()),
        TorontoData.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


## 2. Explore data

I just use geolocator to get the geographical coordinates of Toronto to plot the information using Folium

In [80]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [81]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = TorontoData

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                           neighborhoods['Longitude'],
                                           neighborhoods['Borough'],
                                           neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Configure Foursquare access

In [82]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'yyy' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version


I make use of a function that takes the names and locations of the neighborhoods in Toronto and obtains the 100 top venues around.

In [83]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in tqdm(zip(names, latitudes, longitudes), total = names.size):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
Toronto_venues = getNearbyVenues(TorontoData.Neighbourhood,
                            TorontoData.Latitude,
                            TorontoData.Longitude)

100%|██████████| 38/38 [00:12<00:00,  4.28it/s]


I print the shape of my data and also a sample of the initial rows of information

In [85]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1712, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
1,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
2,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
3,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant
4,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Big Crow,43.675896,-79.403680,BBQ Joint


I can check the amount of venues per neighboorhood

In [86]:
Toronto_venues.groupby("Neighborhood").Venue.count().sort_values(ascending=False).head()

Neighborhood
Adelaide, King, Richmond                     100
St. James Town                               100
Ryerson, Garden District                     100
Chinatown, Grange Park, Kensington Market    100
Commerce Court, Victoria Hotel               100
Name: Venue, dtype: int64

And the total amount of unique categories in my data

In [87]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


Let us now take the venue category information and create a dataframe with a one hot enconding of these data.

In [90]:
Toronto_OHE = pd.get_dummies(Toronto_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_OHE["Neighborhood"] = Toronto_venues["Neighborhood"]


nindex = list(Toronto_OHE.columns).index("Neighborhood")
cols = deque(Toronto_OHE.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_OHE = Toronto_OHE[cols]

Toronto_OHE.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,...,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue
0,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


As there are 276 unique categories, the datagrame contains all the neighborhoods and 271 columns

In [91]:
Toronto_OHE.shape

(1712, 240)

I can compute the average number of venue categories per neighborhood

In [92]:
Toronto_grouped = Toronto_OHE.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,...,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue
0,"Adelaide, King, Richmond",0.01,0.000000,0.01,0.01,0.01,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0
1,Berczy Park,0.00,0.017241,0.00,0.00,0.00,0.0,0.0,0.017241,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.017241,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.00,0.045455,0.00,0.00,0.00,0.0,0.0,0.000000,0.045455,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.000000,0.00,0.00,0.00,0.0,0.0,0.058824,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.058824,0.000000,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.000000,0.00,0.00,0.00,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0


In total I have 99 Neighborhoods, grouped by borough, and 271 distinct categories

In [93]:
Toronto_grouped.shape

(38, 240)

The number of Neighborhoods obtained from the venues is smaller than the original number of postcodes, what means that in some postcodes there are no venues

For neighborhood clustering let us select only the N most frequent venue categories per neighborhood

In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Steakhouse,Thai Restaurant,Café,Sushi Restaurant,American Restaurant,Asian Restaurant,Bakery,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Steakhouse,Restaurant,Pub,Bakery,Cheese Shop,Farmers Market,Seafood Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Burrito Place,Bar,Gym,Grocery Store,Convenience Store,Falafel Restaurant,Yoga Studio
3,Business Reply Mail Processing Centre 969 Eastern,Burrito Place,Pizza Place,Auto Workshop,Spa,Restaurant,Recording Studio,Comic Shop,Light Rail Station,Smoke Shop,Fast Food Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Airport Gate,Airport,Harbor / Marina,Boat or Ferry,Plane,Airport Food Court,Boutique


## 3. Neighborhood clustering

Before clustering, let us use a PCA decomposition to reduce the noise of the signal and improve clustering efficiency

In [96]:
pca = PCA(.95)
Toronto_grouped_clustering = pca.fit_transform(Toronto_grouped.drop('Neighborhood', 1))
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

The dimensionality reduction has kept only a reduced number of the total dimensions

In [97]:
Toronto_grouped_clustering.shape

(38, 239)

With reduced dimensional dataset we carry out the KMeans clustering

In [98]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[2 2 2 2 2 2 2 2 2 2]
(38,)


Let us now create a dataframe that containes the neighborhood, the location and the cluster information, together with the top 10 venues

In [99]:
Toronto_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
Toronto_combined = TorontoData.merge(Toronto_grouped, left_on = "Neighbourhood", right_on = "Neighborhood", how = "outer")

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_combined = Toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_combined["Cluster Labels"] = Toronto_combined["Cluster Labels"].fillna(5).astype("int")

Toronto_combined.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,"The Annex, North Midtown, Yorkville",0.0,0.0,0.0,0.000000,...,Café,Sandwich Place,Coffee Shop,Pizza Place,Park,Liquor Store,Convenience Store,Cosmetics Shop,Pub,American Restaurant
1,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Central Bay Street,0.0,0.0,0.0,0.011765,...,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Bar,Middle Eastern Restaurant,Sushi Restaurant,Salad Place,Sandwich Place
2,M4S,Central Toronto,Davisville,43.704324,-79.388790,Davisville,0.0,0.0,0.0,0.000000,...,Dessert Shop,Sandwich Place,Restaurant,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Pizza Place,Japanese Restaurant,Seafood Restaurant
3,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0,0.0,0.000000,...,Airport Lounge,Airport Terminal,Airport Service,Airport Gate,Airport,Harbor / Marina,Boat or Ferry,Plane,Airport Food Court,Boutique
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,North Toronto West,0.0,0.0,0.0,0.000000,...,Sporting Goods Shop,Coffee Shop,Dessert Shop,Pet Store,Health & Beauty Service,Grocery Store,Salon / Barbershop,Fast Food Restaurant,Clothing Store,Yoga Studio


In [100]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_combined['Latitude'],
                                  Toronto_combined['Longitude'],
                                  Toronto_combined['Neighborhood'],
                                  Toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Analyze clusters

## Cluster 1

Cluster 1 contains many restaurants and diners, as well as some other services, like bus and train stations

In [101]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 0, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Park,Playground,Trail,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Music Venue


## Cluster 2

Cluster 2 seems to be dominated by banks, music venues and airport facilities

In [102]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 1, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Gym,Playground,Music Venue,Accessories Store,Art Museum,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal


## Cluster 3

Cluster 3 is mostly linked to open areas for sport and children

In [103]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 2, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Café,Sandwich Place,Coffee Shop,Pizza Place,Park,Liquor Store,Convenience Store,Cosmetics Shop,Pub,American Restaurant
1,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Bar,Middle Eastern Restaurant,Sushi Restaurant,Salad Place,Sandwich Place
2,Dessert Shop,Sandwich Place,Restaurant,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Pizza Place,Japanese Restaurant,Seafood Restaurant
3,Airport Lounge,Airport Terminal,Airport Service,Airport Gate,Airport,Harbor / Marina,Boat or Ferry,Plane,Airport Food Court,Boutique
4,Sporting Goods Shop,Coffee Shop,Dessert Shop,Pet Store,Health & Beauty Service,Grocery Store,Salon / Barbershop,Fast Food Restaurant,Clothing Store,Yoga Studio


## Cluster 4

Cluster 4 is the most homogeneous one (it only contains two neighborhoods) and seems to be dominated by the Baseball field

In [104]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 3, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Garden,Home Service,Music Venue,Accessories Store,Art Gallery,Aquarium,Antique Shop,American Restaurant,Airport Terminal,Airport Service


## Cluster 5

Cluster 5 is characterized by shops and restaurants

In [105]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 4, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Mexican Restaurant,Jewelry Store,Trail,Sushi Restaurant,Music Venue,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service
